In [1]:
!git clone https://github.com/nothings/stb.git

Cloning into 'stb'...
remote: Enumerating objects: 7851, done.
remote: Counting objects: 100% (580/580), done.
remote: Compressing objects: 100% (215/215), done.
remote: Total 7851 (delta 376), reused 547 (delta 365), pack-reused 7271
Receiving objects: 100% (7851/7851), 5.45 MiB | 4.67 MiB/s, done.
Resolving deltas: 100% (5221/5221), done.


In [2]:
!ls
!mkdir Cuda_Image
!mv /content/stb/stb_image*.* Cuda_Image

sample_data  stb


In [3]:
!wget -O skyline.jpg https://images-prod-uitdatabank.imgix.net/329eb7c3-df7c-413d-9196-955fd13526af.jpeg
%cd Cuda_Image
!ls
# c/p stb_image.h  stb_image_resize.h  stb_image_write.h to your project directory

--2022-09-26 19:04:55--  https://images-prod-uitdatabank.imgix.net/329eb7c3-df7c-413d-9196-955fd13526af.jpeg
Resolving images-prod-uitdatabank.imgix.net (images-prod-uitdatabank.imgix.net)... 146.75.38.208, 2a04:4e42:79::720
Connecting to images-prod-uitdatabank.imgix.net (images-prod-uitdatabank.imgix.net)|146.75.38.208|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2427278 (2.3M) [image/jpeg]
Saving to: ‘skyline.jpg’

skyline.jpg         100%[===================>]   2.31M  --.-KB/s    in 0.08s   

2022-09-26 19:04:55 (30.9 MB/s) - ‘skyline.jpg’ saved [2427278/2427278]

/content/Cuda_Image
stb_image.h  stb_image_resize.h  stb_image_write.h


In [4]:
%%file image_read.c

#include <stdio.h>
#include <stdlib.h>
#define STB_IMAGE_IMPLEMENTATION
#include "stb_image.h"
#define STB_IMAGE_WRITE_IMPLEMENTATION
#include "stb_image_write.h"

int main(void)
{
    int width, height, channels;
    unsigned char *img = stbi_load("skyline.jpg", &width, &height, &channels, 0);
    if(img == NULL) {
          printf("Error in loading the image\n");
          exit(1);
     }
    printf("Loaded image with a width of %dpx, a height of %dpx and %d channels\n", width, height, channels);
    return 0;
}

Writing image_read.c


In [5]:
!gcc -o -Wall -o image_read image_read.c -lm

In [6]:
!./image_read

Error in loading the image


In [7]:
%%file image_read_write_jpg2png.c

#include <stdio.h>
#include <stdlib.h>
#define STB_IMAGE_IMPLEMENTATION
#include "stb_image.h"
#define STB_IMAGE_WRITE_IMPLEMENTATION
#include "stb_image_write.h"

int main(void)
{
    int width, height, channels;
    unsigned char *img = stbi_load("../skyline.jpg", &width, &height, &channels, 0);
    if(img == NULL) {
          printf("Error in loading the image\n");
          exit(1);
     }
    printf("Loaded image with a width of %dpx, a height of %dpx and %d channels\n", width, height, channels);
    stbi_write_png("skyline.png",width,height,channels,img,width*channels);
    stbi_image_free(img);
    return 0;
}

Writing image_read_write_jpg2png.c


In [8]:
!gcc -o -Wall -o image_read_write_jpg2png image_read_write_jpg2png.c -lm
!./image_read_write_jpg2png

Loaded image with a width of 3648px, a height of 2736px and 3 channels


In [9]:
%%file image_jpg_color2gray.c

#include <stdio.h>
#include <stdlib.h>
#define STB_IMAGE_IMPLEMENTATION
#include "stb_image.h"
#define STB_IMAGE_WRITE_IMPLEMENTATION
#include "stb_image_write.h"

int main(void)
{
    int width, height, channels;
    unsigned char *img = stbi_load("../skyline.jpg", &width, &height, &channels, 0);
    if(img == NULL) {
          printf("Error in loading the image\n");
          exit(1);
     }
    printf("Loaded image with a width of %dpx, a height of %dpx and %d channels\n", width, height, channels);
    
    // Convert the input image to gray
    size_t img_size = width * height * channels;
    int gray_channels = channels == 4 ? 2 : 1;
    size_t gray_img_size = width * height * gray_channels;

    unsigned char *gray_img = malloc(gray_img_size);
    if(gray_img == NULL) {
        printf("Unable to allocate memory for the gray image.\n");
        exit(1);
    }

    for(unsigned char *p = img, *pg = gray_img; p != img + img_size; p += channels, pg += gray_channels) {
        *pg = (uint8_t)((*p + *(p + 1) + *(p + 2))/3.0);
        if(channels == 4) {
            *(pg + 1) = *(p + 3);
        }
    }
    stbi_write_jpg("skyline_gray.jpg", width, height, gray_channels, gray_img, 100);
    stbi_image_free(img);
    return 0;
}

Writing image_jpg_color2gray.c


In [10]:
!gcc -o -Wall -o image_jpg_color2gray image_jpg_color2gray.c -lm
!./image_jpg_color2gray

Loaded image with a width of 3648px, a height of 2736px and 3 channels


In [11]:
%%timeit
!./image_jpg_color2gray

Loaded image with a width of 3648px, a height of 2736px and 3 channels
Loaded image with a width of 3648px, a height of 2736px and 3 channels
Loaded image with a width of 3648px, a height of 2736px and 3 channels
Loaded image with a width of 3648px, a height of 2736px and 3 channels
Loaded image with a width of 3648px, a height of 2736px and 3 channels
Loaded image with a width of 3648px, a height of 2736px and 3 channels
Loaded image with a width of 3648px, a height of 2736px and 3 channels
Loaded image with a width of 3648px, a height of 2736px and 3 channels
1.62 s ± 2.35 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
